In [36]:
import os

old_folder = r'D:\Pu\20220204-P_brain_M1_nonclear\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Pu\20220209-P_brain_CTP11-1000_CTP12_from_0204\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

In [37]:
os.path.isfile(before_position_file), os.path.isfile(after_position_file)

(True, True)

# Calculate rotation matrix

In [38]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [39]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)
R, T

(array([[ 0.99996207, -0.00870944],
        [ 0.00870944,  0.99996207]]),
 array([  51.86510187, -472.89365466]))

# Translate 60x positions

In [40]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [41]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-4945.21040471  -540.17449538]
 [-4946.95229198  -740.16690981]
 [-4948.69417926  -940.15932424]
 [-4950.43606653 -1140.15173867]
 [-4952.1779538  -1340.14415309]
 [-4752.18553937 -1341.88604037]
 [-4750.4436521  -1141.89362594]
 [-4748.70176483  -941.90121151]
 [-4746.95987756  -741.90879708]
 [-4745.21799029  -541.91638265]
 [-4545.22557586  -543.65826992]
 [-4546.96746313  -743.65068435]
 [-4548.7093504   -943.64309878]
 [-4550.45123767 -1143.63551321]
 [-4552.19312494 -1343.62792764]
 [-4353.94259779 -1545.36222934]
 [-4352.20071051 -1345.36981491]
 [-4350.45882324 -1145.37740048]
 [-4348.71693597  -945.38498605]
 [-4346.9750487   -745.39257162]
 [-4345.23316143  -545.4001572 ]
 [-4143.49885973  -347.14963004]
 [-4145.240747    -547.14204447]
 [-4146.98263427  -747.1344589 ]
 [-4148.72452154  -947.12687332]
 [-4150.46640881 -1147.11928775]
 [-4152.20829609 -1347.11170218]
 [-4153.95018336 -1547.10411661]
 [-3953.95776893 -1548.84600388]
 [-3952.21588166 -1348.85358945]
 [-3950.47

In [42]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220209-P_brain_CTP11-1000_CTP12_from_0204\Alignment\translated_positions_all.txt


# Manual adjust positions

In [48]:
manual_positions_before_file = os.path.join(os.path.dirname(old_folder), '60x_positions_before.txt')
print(manual_positions_before_file)

D:\Pu\20220204-P_brain_M1_nonclear\60x_positions_before.txt


In [47]:
manual_positions = np.loadtxt(manual_positions_before_file, delimiter=',')
print(manual_positions)
translated_manual_positions = np.dot(manual_positions, R) + T
print('translated manual positions:')
print(np.round(translated_manual_positions, 2))

[[-5105.14 -1634.23]
 [-5518.84  2938.95]
 [  220.37  3695.69]
 [ 1613.92  -771.13]]
translated manual positions:
[[-5067.31 -2062.6 ]
 [-5441.17  2514.01]
 [  304.41  3220.74]
 [ 1659.01 -1258.05]]


In [52]:
manual_real_positions = [
    [-5062.7, -2070.5],
    [-5446, 2508],
    [303, 3221.05],
    [1662.5, -1255.3],
]
manual_shifts = np.array(manual_real_positions) - translated_manual_positions[:len(manual_real_positions)]
manual_shifts

array([[ 4.61449327, -7.90122011],
       [-4.83101764, -6.010863  ],
       [-1.41412055,  0.31312276],
       [ 3.49221832,  2.75074087]])

In [53]:
manual_shift = np.mean(manual_shifts, axis=0)
manual_shift

array([ 0.46539335, -2.71205487])

In [54]:
adjusted_new_positions = new_positions + manual_shift

In [55]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220209-P_brain_CTP11-1000_CTP12_from_0204\Alignment\adjusted_translated_positions_all.txt
